In [ ]:
from flask import Flask, request, send_file, render_template_string
from Crypto.Cipher import DES
from io import BytesIO
from pyngrok import conf, ngrok

app = Flask(__name__)

# Gán mã ngrok token
conf.get_default().auth_token = "2x78kyYgoItDVATusfDXfO8EB92_7BsHuMDhvAYST2gRrDws2"

# Hàm pad dữ liệu đủ bội số 8 bytes (block DES)
def pad(data: bytes) -> bytes:
    while len(data) % 8 != 0:
        data += b' '
    return data

# Hàm mã hóa file dữ liệu
def encrypt_file(data: bytes, key: str) -> bytes:
    if len(key) != 8:
        raise ValueError("Khóa phải có đúng 8 ký tự.")
    cipher = DES.new(key.encode(), DES.MODE_ECB)
    encrypted = cipher.encrypt(pad(data))
    return encrypted

# Hàm giải mã file dữ liệu
def decrypt_des(data: bytes, key: str) -> bytes:
    if len(key) != 8:
        raise ValueError("Khóa phải có đúng 8 ký tự.")
    cipher = DES.new(key.encode(), DES.MODE_ECB)
    decrypted = cipher.decrypt(data)
    return decrypted.rstrip(b' ')

# HTML giao diện web
HTML = '''
<!DOCTYPE html>
<html>
<head>
    <title>DES Encryption/Decryption</title>
    <style>
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background: #eef2f3;
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
        }

        .container {
            background: #fff;
            padding: 30px;
            border-radius: 10px;
            box-shadow: 0 0 15px rgba(0,0,0,0.2);
            width: 400px;
        }

        h1 {
            text-align: center;
            color: #333;
        }

        form {
            display: flex;
            flex-direction: column;
        }

        input[type="file"],
        input[type="text"],
        select,
        input[type="submit"] {
            margin: 10px 0;
            padding: 10px;
            font-size: 16px;
            border: 1px solid #ccc;
            border-radius: 6px;
        }

        input[type="submit"] {
            background-color: #4CAF50;
            color: white;
            font-weight: bold;
            cursor: pointer;
            transition: background-color 0.3s;
        }

        input[type="submit"]:hover {
            background-color: #45a049;
        }

        .error {
            color: red;
            font-weight: bold;
            text-align: center;
        }

        .success {
            color: green;
            font-weight: bold;
            text-align: center;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>DES Encrypt/Decrypt</h1>
        {% if error %}
            <p class="error">{{ error }}</p>
        {% endif %}
        <form method="post" enctype="multipart/form-data" onsubmit="return validateForm()">
            <input type="file" name="file" id="file" required>
            <input type="text" name="key" id="key" placeholder="8-character key" maxlength="8" required>
            <select name="action">
                <option value="encrypt">Encrypt</option>
                <option value="decrypt">Decrypt</option>
            </select>
            <input type="submit" value="Submit">
        </form>
        <p id="form-error" class="error" style="display:none;"></p>
    </div>

    <script>
        function validateForm() {
            const key = document.getElementById("key").value;
            const file = document.getElementById("file").value;
            const errorElement = document.getElementById("form-error");

            if (!file) {
                errorElement.textContent = "Please select a file.";
                errorElement.style.display = "block";
                return false;
            }

            if (key.length !== 8) {
                errorElement.textContent = "Key must be exactly 8 characters.";
                errorElement.style.display = "block";
                return false;
            }

            errorElement.style.display = "none";
            return true;
        }
    </script>
</body>
</html>


'''


@app.route('/', methods=['GET', 'POST'])
def home():
    error = None
    if request.method == 'POST':
        f = request.files['file']
        key = request.form['key']
        mode = request.form['mode']
        data = f.read()

        try:
            if mode == 'encrypt':
                result = encrypt_file(data, key)
                filename = f"ma_hoa_{f.filename}"
            else:
                result = decrypt_des(data, key)
                filename = f"giai_ma_{f.filename}"

            buffer = BytesIO(result)
            buffer.seek(0)
            return send_file(buffer, as_attachment=True, download_name=filename)
        except Exception as e:
            error = f"Lỗi: {e}"

    return render_template_string(HTML, error=error)

# Khởi tạo ngrok
public_url = ngrok.connect(5000)
print(f"📌 Truy cập web tại: {public_url}")

# Chạy Flask trên Google Colab
app.run(port=5000, use_reloader=False)

📌 Truy cập web tại: NgrokTunnel: "https://9202-35-204-205-188.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [15/May/2025 11:40:41] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/May/2025 11:40:42] "GET /favicon.ico HTTP/1.1" 404 -


In [ ]:
!pip install pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 36.0 MB/s eta 0:00:00


In [ ]:
!pip install flask pyngrok pycryptodome